In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import resnet50
from torch.utils.tensorboard import SummaryWriter

import pickle

In [3]:
# x_train
!gdown 158wGhDi741wVLY6QFY5G_ti6Gsm9Ygqq

# y_train
!gdown 13_XDA8AGCnOTQ_J1NFwz9LVduQEdG_9r

# x_test
!gdown 1d350cfCFVL6F146lO8AF5DQYauAFfMHI

# y_test
!gdown 14ygmP6cqQd51s-Hklo4QHubatkR4p9ex

Downloading...
From (original): https://drive.google.com/uc?id=158wGhDi741wVLY6QFY5G_ti6Gsm9Ygqq
From (redirected): https://drive.google.com/uc?id=158wGhDi741wVLY6QFY5G_ti6Gsm9Ygqq&confirm=t&uuid=60ea8cd0-fc62-4bb6-8b1a-4f094077e1f4
To: /content/x_train.pickle
100% 779M/779M [00:10<00:00, 76.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=13_XDA8AGCnOTQ_J1NFwz9LVduQEdG_9r
To: /content/y_train.pickle
100% 41.5k/41.5k [00:00<00:00, 50.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1d350cfCFVL6F146lO8AF5DQYauAFfMHI
From (redirected): https://drive.google.com/uc?id=1d350cfCFVL6F146lO8AF5DQYauAFfMHI&confirm=t&uuid=990189cc-f78e-4c67-b3d5-4adf3f3aae50
To: /content/x_test.pickle
100% 260M/260M [00:04<00:00, 55.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=14ygmP6cqQd51s-Hklo4QHubatkR4p9ex
To: /content/y_test.pickle
100% 14.0k/14.0k [00:00<00:00, 33.4MB/s]


In [4]:
with open('x_train.pickle', 'rb') as f:
    x_train = pickle.load(f)

with open('y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)

with open('x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)

with open('y_test.pickle', 'rb') as f:
    y_test = pickle.load(f)

In [5]:
x_train.shape, y_train.shape

((5174, 224, 224, 3), (5174,))

In [6]:
x_test.shape, y_test.shape

((1725, 224, 224, 3), (1725,))

In [16]:
writer = SummaryWriter('runs/experiment')

In [28]:
# 데이터 전처리
def preprocess_data(images, labels):
    # int8 -> float32로 변환 및 정규화 (-127.5~127.5 -> -1~1)
    images = torch.FloatTensor(images) / 127.5
    # 차원 변환: b(0), h(1), w(2), c(3) -> b(0), c(3), h(1), w(2)
    images = images.permute(0, 3, 1, 2)
    labels = torch.LongTensor(labels)
    return images, labels

In [27]:
    # resnet 사전학습 모델 불러오기 + head 수정
def get_resnet50(num_classes):
    model = resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    # 콜백 : early stopping, lr schedule, model saving, tensorboard




TensorDataset을 사용하는 것을 권장(필수는 아님):

- 데이터와 레이블이 쌍으로 관리됨
- 데이터 변환이나 증강을 적용하기 쉬움
- 여러 종류의 데이터를 함께 관리할 때 유용
- 인덱싱이나 슬라이싱이 편리

In [19]:
# 데이터 준비
train_images, train_labels = preprocess_data(x_train, y_train)
train_dataset = TensorDataset(train_images, train_labels)

In [20]:
train_images.shape, train_labels.shape

(torch.Size([5174, 3, 224, 224]), torch.Size([5174]))

In [22]:
# 데이터 분할 (90% 학습, 10% 검증)
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [23]:
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [24]:
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_classes = len(torch.unique(train_labels))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.0001)

NameError: name 'resnet' is not defined

In [25]:
# ResNet50 학습
print("\nTraining ResNet50...")
resnet = get_resnet50(num_classes).to(device)

train_model(resnet, train_loader, val_loader, criterion, optimizer,
            num_epochs=3, device=device)

writer.close()# 텐서보드 설정


Training ResNet50...


NameError: name 'model' is not defined

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs